# Pageviews for W:PM and W pages with external links
- counts exclude bots/spiders (agent_type = 'user')

In [1]:
# basic defaults, including study dates, common SQL exclusions and parquet files for anonymized data
%run -i 'data-defaults.py'
# pandas float formatting
pd.options.display.float_format = '{:.4f}'.format

## Total pageviews for WP:M pages with external links

In [2]:
wpm_query = """
SELECT access_method, SUM(view_count) AS total_pageviews, SUM(view_count)/{} AS daily_average_pageviews
FROM wmf.pageview_hourly
WHERE project = 'en.wikipedia'
AND page_id IN
    (SELECT DISTINCT page_id 
    FROM ryanmax.population_wpm_pages_with_extlinks)
AND agent_type = 'user'
AND to_date(CONCAT(year,'-',month,'-',day)) >= '{}'
AND to_date(CONCAT(year,'-',month,'-',day)) <= '{}'
GROUP BY access_method
"""
wpm_pageviews = spark.sql(wpm_query.format(days_in_study,start_date_string, end_date_string))
wpm_pageviews_rdd = wpm_pageviews.rdd
wpm_pageviews_df = sqlContext.createDataFrame(wpm_pageviews_rdd)
wpm_pageviews_pandas = wpm_pageviews_df.toPandas()

wpm_pageviews_pandas

,access_method,total_pageviews,daily_average_pageviews
0,desktop,62650292,1957821.6250
1,mobile web,124214376,3881699.2500
2,mobile app,1150383,35949.4688


## Total pageviews for W pages with external links

In [3]:
w_query = """
SELECT access_method, SUM(view_count) AS total_pageviews, SUM(view_count)/{} AS daily_average_pageviews
FROM wmf.pageview_hourly
WHERE project = 'en.wikipedia'
AND page_id IN
    (SELECT DISTINCT page_id 
    FROM ryanmax.population_w_pages_with_extlinks)
AND agent_type = 'user'
AND to_date(CONCAT(year,'-',month,'-',day)) >= '{}'
AND to_date(CONCAT(year,'-',month,'-',day)) <= '{}'
GROUP BY access_method
"""
w_pageviews = spark.sql(w_query.format(days_in_study,start_date_string, end_date_string))
w_pageviews_rdd = w_pageviews.rdd
w_pageviews_df = sqlContext.createDataFrame(w_pageviews_rdd)
w_pageviews_pandas = w_pageviews_df.toPandas()

w_pageviews_pandas

,access_method,total_pageviews,daily_average_pageviews
0,desktop,3134600739,97956273.0938
1,mobile web,4120797860,128774933.1250
2,mobile app,54845509,1713922.1562


## Top WP:M pages

In [5]:
# most visited WP:M pages with external links
top_wpm_query = """
SELECT page_id, page_title, SUM(view_count) AS total_pageviews, SUM(view_count)/{} AS daily_average_pageviews
FROM wmf.pageview_hourly
WHERE project = 'en.wikipedia'
AND page_id IN
    (SELECT DISTINCT page_id 
    FROM ryanmax.population_wpm_pages_with_extlinks)
AND agent_type = 'user'
AND to_date(CONCAT(year,'-',month,'-',day)) >= '{}'
AND to_date(CONCAT(year,'-',month,'-',day)) <= '{}'
GROUP BY page_id, page_title
ORDER BY total_pageviews DESC
LIMIT 50
"""

top_wpm = spark.sql(top_wpm_query.format(days_in_study,start_date_string, end_date_string))
top_wpm.limit(50).toPandas()

,page_id,page_title,total_pageviews,daily_average_pageviews
0,43573275,Elizabeth_Holmes,1349128,42160.2500
1,41779862,Theranos,546943,17091.9688
2,58911,Measles,386040,12063.7500
3,18079,Leonardo_da_Vinci,372784,11649.5000
4,27546,Sexual_intercourse,346756,10836.1250
5,37556,Asperger_syndrome,321686,10052.6875
6,791546,Ketogenic_diet,317814,9931.6875
7,56880920,Ramesh_Balwani,305086,9533.9375
8,1232575,Suicide_methods,280846,8776.4375
9,44990,Ryan_White,260539,8141.8438
